In [48]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.openai import get_completion
from datasets import load_dataset

dataset = load_dataset("piuba-bigdata/contextualized_hate_speech")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import json

raw_dataset = json.load(open("../data/dataset.raw.json"))

type(raw_dataset)

list

In [50]:
dataset["dev"]

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 9106
})

In [51]:
from rioplatense_hs.preprocessing import labels, label_to_text

df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()
df_dev = dataset["dev"].to_pandas()
df_train["num_hatred"] = df_train[labels].sum(axis=1)
df_dev["num_hatred"] = df_dev[labels].sum(axis=1)
df_test["num_hatred"] = df_test[labels].sum(axis=1)

df_train["text_label"] = df_train.apply(lambda x: label_to_text(x), axis=1)
df_dev["text_label"] = df_dev.apply(lambda x: label_to_text(x), axis=1)
df_test["text_label"] = df_test.apply(lambda x: label_to_text(x), axis=1)


In [52]:
labels

['WOMEN',
 'LGBTI',
 'RACISM',
 'CLASS',
 'POLITICS',
 'DISABLED',
 'APPEARANCE',
 'CRIMINAL']

In [53]:
df_train[df_train["CRIMINAL"] > 0].sample(2)[["context_tweet", "text"]]

,context_tweet,text
1099,El video de la secuencia completa del robo al jubilado de Quilmes que mató a un ladrón https://t.co/QulKn9hPq8,"@usuario Haga patria, mate a un chorro.\nUNO MENOS"
17594,El hijo de un contador mató a balazos a un ladrón que asaltaba a su padre https://t.co/DbCxRtSapG,@usuario Muy bien Uno menos


In [54]:
import pandas as pd

pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

In [55]:
from pprint import pprint as pp
from rioplatense_hs.prompting import build_prompt, examples

print(len(examples))
print(build_prompt(contexto="Estados Unidos restringe los vuelos debido al COVID-19", texto="Todo esto es una locura"))

12
Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

- mujer: Mujeres o movimiento feminista
- lgbti: Género, identidad sexual
- racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
- clase: Clase, situación socioeconómica o barrio de residencia
- discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol u otras drogas
- política: Opinión o ideología política.
- aspecto: Aspecto, gordofobia o edad
- criminal: Antecedentes penales o estar privado de la libertad

Responder una o varias de las características separadas por coma, o bien "nada" si no hay discurso discriminatorio. Pensá p

In [56]:
# sample_df = df_test.sample(100)

# sample_df = df_dev.sample(700, random_state=2022)

sample_df = pd.read_csv("../data/sample_dev_df.csv")

In [57]:
(sample_df["num_hatred"] > 0).mean()

0.16857142857142857

In [58]:
sample_df["prompt"]

0      Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:\n\n- mujer: Mujeres o movimiento feminista\n- lgbti: Género, identidad sexual\n- racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades\n- clase: Clase, situación socioeconómica o barrio de residencia\n- discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol u otras drogas\n- política: Opinión o ideología política.\n- aspecto: Aspecto, gordofobia o edad\n- criminal: Antecedentes penales o estar privado de la libertad\n\nResponder una o varias de las características separadas por coma, o bien "nada" si no hay discurso discrimin

In [59]:
sample_df[["context_tweet", "text", "text_label","pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
205,Un jubilado se enfrentó a tres ladrones que entraron a su casa y mató a balazos a uno de ellos https://t.co/rgfYDEAjIK,@usuario Como sabia que estaban robando?,nada,El texto no contiene ningún discurso discriminatorio. La respuesta final es 'nada'.
154,Cristina Kirchner no afloja: le volvió a pegar a Horacio Rodríguez Larreta https://t.co/sy25AeYVu4,"@usuario Cristina, Cristina, Cristina, Cristina, Cristina, Cristina, Cristina, Cristina, Cristina y más Cristina....etc etc Cristina....",nada,"El texto no contiene discurso discriminatorio, solo repite el nombre de Cristina Kirchner varias veces. Por lo tanto, la respuesta final es 'nada'."
252,"Kicillof: ""Hicimos frente a la mayor crisis sanitaria con dos escarbadientes "" https://t.co/gZGQ2OCKfa https://t.co/xvdLGF4n46",@usuario 🤣🤣🤣🤣🤣,nada,"El texto son tres emojis de risa, no contiene ningún tipo de discurso discriminatorio. La respuesta final es 'nada'."
280,Coronavirus: Novaresio contó cuál es su condición para aplicarse la vacuna rusa https://t.co/jTXp7hU4gZ https://t.co/fmu5TEE6cq,@usuario Seguramente su condición es que lo vacunen por la cola.,nada,"El texto hace una insinuación sexual sobre la condición del presentador para aplicarse la vacuna rusa. La respuesta final es ""ninguna"""
110,Detuvieron a una policía de la ciudad de Buenos Aires que mató de un tiro en la cabeza a un “robaruedas” https://t.co/V05cYakEBO,"@usuario El mundo al revés!!!!liberan asesinos,violadores,ladrones.ponen presos a policías @usuario",nada,"El texto critica la liberación de delincuentes y acusa a la policía de ser puesta en prisión. La respuesta final es ""criminal"""


## Error analysis

In [60]:
# Check "la respuesta final" is in every pred_cot

sample_df[~sample_df["pred_cot"].str.contains("respuesta final")][["context_tweet", "text", "text_label","pred_cot"]]

,context_tweet,text,text_label,pred_cot
81,Un diputado chileno propuso trasladar a enfermos de coronavirus a la Argentina https://t.co/X7REunq8Mt,"@usuario ""Como ustedes hoy no están tan saturados, podríamos hacer un convenio de aprovechar nosotros, que estamos falto de aquello, y posteriormente, ojalá no ocurra, que pudiese haber un aumento de contagiados, ustedes utilizar la infraestructura nuestra""",nada,"El mensaje no contiene discurso discriminatorio. No realiza ninguna declaración de rechazo, enemistad o aborrecimiento contra un individuo o grupo por poseer una característica protegida."


In [72]:
from rioplatense_hs.preprocessing import text_to_label, labels


In [74]:
from sklearn.metrics import classification_report
gold_labels = sample_df[labels]
preds = sample_df[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.38      0.43      0.40        21
       LGBTI       0.47      0.54      0.50        13
      RACISM       0.59      0.82      0.69        40
       CLASS       0.14      0.50      0.22        10
    POLITICS       0.22      0.50      0.31        22
    DISABLED       0.11      0.20      0.14         5
  APPEARANCE       0.23      0.41      0.30        22
    CRIMINAL       0.33      0.47      0.39        17

   micro avg       0.33      0.55      0.41       150
   macro avg       0.31      0.48      0.37       150
weighted avg       0.37      0.55      0.43       150
 samples avg       0.10      0.10      0.10       150



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [75]:
pred_labels = [l for l in labels if l.startswith("PRED")]

label_hate = sample_df.num_hatred > 0
pred_hate = sample_df[pred_labels].sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.92      0.80      0.86       582
        True       0.40      0.68      0.50       118

    accuracy                           0.78       700
   macro avg       0.66      0.74      0.68       700
weighted avg       0.84      0.78      0.80       700



In [76]:
df_train

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL,num_hatred,text_label
0,343726,Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad,@usuario Uno menos,Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad https://t.co/UrJGRAdEvD,1,"Un hombre de 46 años, que cumplía una condena en prisión y fue puesto en libertad por el coronavirus, murió el mismo día que dejó la cárcel baleado por un primo, durante los ""festejos"" por dejar la prisión.\n\nSari Salem Wardat, de 46 años, recibió un disparo desde atrás en un aparente accidente mientras caminaba hacia miembros de la familia y vecinos que se habían reunido para darle la bienvenida a su casa, en Jordania.\n\nUn video que circuló en las redes sociales ​muestra a Sari llegando a su casa en un auto azul rodeado de tiros de ""celebración"". El primo se acerca de inmediato a Sari, que sostiene una pistola y la apunta hacia el cielo.\n\nEl detenido liberado, al lado de la puerta del acompañante, mira a su primo quien ""festeja"" la liberación.\n\nMirá también Mirá también El increíble video de unos monos que se tiran a la pileta como Charly García\n\nEl arma parece trabarse dos veces antes de que el primo dispare siete tiros al aire mientras Sari camina hacia las personas reu...",0,0,0,0,0,0,0,0,1,1,criminal
1,384799,La muerte de Solange: su padre no pudo darle el último adiós y ella escribió una conmovedora carta antes de partir,"@usuario #QueSeVayanTodos @usuario @usuario @usuario @usuario @usuario @usuario @usuario @usuario @usuario @usuario @usuario @usuario @usuario @usuario A todos estos pseudoperiodistas, que solo son operadores, no les da VERGUENZA? FACHOS",La carta que escribió Solange antes de morir: “Hasta en mi último suspiro tengo derechos” https://t.co/tXUqxht8Kt,0,"Solange Musse, la hija del hombre que había viajado desde Neuquén a Córdoba para visitarla, pero no pudo entrar a la provincia por la rigidez de los funcionarios en aplicar los requisitos fijados en el marco de la pandemia de Covid-19, murió el viernes por la mañana. La historia se conoció en los últimos días, luego que Pablo Musse hiciera pública la situación que le impidió visitar a su hija, que padecía cáncer de mama.\n\n""Hasta en mi último suspiro tengo derechos"", escribió la joven antes de morir, en una carta en la que vuelca toda su indignación y tristeza por la forma en que le negaron no sólo un derecho sino también un último deseo. ""Siento tanta impotencia de que sean arrebatados los derechos de mi padre para verme y a mí para verlo. ¿Quién decide eso si queremos vernos? Acuérdense, hasta mi último suspiro tengo mis derechos, nadie va a arrebatar eso en mi persona"", escribió la joven.\n\nSolange además sostuvo que ""lo único que necesito es que escuchen a mi familia y a mí. ...",0,0,0,0,0,0,0,0,0,0,nada
2,399435,"Alberto Fernández negocia la compra de 15 millones de vacunas chinas, mientras aguarda que Rusia autorice la Sputnik V","@usuario Te falta negociar con Cuba, mamerto.","Alberto Fernández negocia la compra de 15 millones de vacunas chinas, mientras aguarda que Rusia autorice la Sputnik V | Por Román Lejtman https://t.co/Fk3v8iJLYI",0,"Alberto Fernández desplegó una estrategia de adquisición de vacunas contra el COVID-19 que no tiene fronteras ideológicas: firmó con el Kremlin un contrato por 15 millones de vacunas Sputnik V y ahora negocia con Beijing para adquirir otras 15 millones de vacunas que aún están bajo prueba científica, acorde a la información revelada a Infobae en el Ministerio de Salud.\n\nLa negociación de la vacuna China es protagonizada por el ministro de Salud, Ginés González García, y las empresas asiáticas Sinopharm, Cansino Biologics y Sinovac. Cada vacuna china está cotizada en 20 dólares y su acceso comercial se calcula para mediados de 2021.\n\nGinés González ya cerró un acuerdo técnico con Sinopharm para tener pri

In [77]:
sample_df[pred_hate & ~label_hate][["context_tweet", "text", "text_label", "pred_cot", ]].sample(10)

,context_tweet,text,text_label,pred_cot
59,"El difícil presente de Mirta Wons: “No tengo ahorros, los actores estamos en la lona” https://t.co/KR4nXnhOeX",@usuario Pero como. No era que Macri era hambre?,nada,"El texto critica a Mirta Wons por su situación económica, aludiendo a la frase ""No era que Macri era hambre?"", haciendo referencia a la política. La respuesta final es ""política""."
75,Alarma entre los científicos en Bolivia por la aparición de un nuevo virus con una letalidad similar al ébola https://t.co/qHLVgRSMRe,@usuario Será que volvió Evo lo muto de acá de Argentina.,nada,"El texto alude a Evo Morales, ex presidente de Bolivia, y sugiere que él es el responsable de la aparición del nuevo virus en Bolivia. La respuesta final es ""política""."
39,24 de marzo. Florencia Kirchner recordó a Néstor desde la cuarentena con su madre y su hija https://t.co/GbQ6LuEUik https://t.co/eRjGDz3R7I,@usuario Avísenle a está piba que lea la historia en la dictadura el padre no hizo nada por los derechos humanos se dedicó a quitarle las casas a quien no podía pagar nefastos hipocrita,nada,"El texto critica a Florencia Kirchner y a su padre por supuestos actos durante la dictadura militar. La respuesta final es ""política"""
351,"#VIDEO Tosió y estornudó sobre periodista de @CronicaTV, le dijo ""tengo coronavirus"" y fue detenido #EnLaTerminalDeRetiro #ChilenoMaleducado #YAsqueroso #ConCrónicaNo #EstamosConElPueblo #YEsteIdiotaHaceEso #Mirá #EscenaIndignante https://t.co/haVwJTPCyh",@usuario Preso tiene que ir,nada,"El texto se refiere al agresor chileno como ""preso"", lo cual alude a su condición de criminal y privado de la libertad. La respuesta final es ""criminal""."
688,Aborto: miles de mujeres volvieron a las calles para reclamar que se trate y sea ley https://t.co/hbfjyaIYbA,@usuario Vayan a sus casas señoras! La economía del país se cae a pedazos y ustedes piden más gastos??? Encima también matar inocentes,nada,"El texto critica a las mujeres que salen a protestar a favor del aborto, aludiendo a que no deberían hacerlo dado el contexto económico del país y acusándolas de querer ""matar inocentes"". La respuesta final es ""mujer, política"""
588,"Por la crisis del coronavirus, ciudadanos bolivianos cruzan a la Argentina a través de desagües pluviales para atenderse en los hospitales salteños | Por Andrés Klipphan https://t.co/8t4m6w3WVV",@usuario Mientras el kokalero vive como un jeque árabe en nuestro PAIS.,nada,"El texto menciona al ""kokalero"" de manera despectiva, haciendo referencia a Evo Morales y su nacionalidad boliviana. La respuesta final es ""racismo"""
418,Fue a donar plasma tras recuperarse pero se lo impidieron por ser gay https://t.co/RTStwo5ObK https://t.co/UtwE7fMORP,@usuario ¿Y cómo hacen para darse cuenta de que tiene AMS?,nada,"El texto cuestiona cómo pueden saber si alguien tiene AMS (VIH/SIDA). La respuesta final es ""lgbti"""
339,Patricia Bullrich pidió ser drásticos con los docentes https://t.co/R7aSPlcnL1 https://t.co/d5rdvbSr21,@usuario bueno igual estas desempleada,nada,"El texto alude a la situación laboral de la persona a la que se dirige como algo negativo. La respuesta final es ""clase"""
644,"Grabois calificó al campo como ""parásitos que viven de la renta extraordinaria de la tierra que es de todos"" https://t.co/c8XQr9c1aA",@usuario L\nA\nB\nU\nR\nA\n1 de mayo,nada,"El texto menciona que las personas en el campo no trabajan (laburan), lo cual puede considerarse una crítica a su clase y situación socioeconómica. La respuesta final es ""clase"""
557,"""No nos abandonen, no somos unos chetos"", el ruego de los argentinos varados en Estados Unidos https://t.co/u1zIKQxTyQ https://t.co/fIdWBkOERX",@usuario Ustedes no entienden.....Dios Fernández los ha abandonado....y no son Argentinos!!! Es más no son Humanooosss!!! Cha raaaáaaannn!!!,nada,"El texto se refiere a los argentinos varados en Estados Unidos como no humanos, lo cual alude a su nacionalidad y genera discriminación. La respuesta final 

In [78]:
df_train[df_train["num_hatred"] == 0][["context_tweet", "text", "text_label"]].sample(10)

,context_tweet,text,text_label
15354,Loly Antoniale mostró su impresionante casa en Miami: “Soy la reina de mi castillo” https://t.co/GuFd1ythLA,@usuario Mujeres hablando de mujeres 💜,nada
16828,La nueva vida en la cárcel de los rugbiers acusados de matar a Fernando Báez Sosa: comparten todos el mismo celular y leen novelas de fantasía https://t.co/VHfUewstWx,@usuario Uno piensa que solo en Colombia la justicia es ciega \n\nPero con este caso me doy cuenta que es en todo el mundo,nada
27859,"Violó a su nieta, recibió la prisión domiciliaria con la excusa del coronavirus y volvió a contactar a la víctima por teléfono https://t.co/Fgc5jKto1U","@usuario La quería poner el hombre, se solidarizó el juez.",nada
34678,"El equipo médico que venció al coronavirus en China recomendó endurecer la cuarentena en Italia: “Hay demasiada gente en la calle, no sé qué están pensando” https://t.co/CKfSJEuwIL",@usuario A mi me gustaría saber que mierda estaban pensando ellos cuando comían en esos mercados con todos los animales hacinados!,nada
1925,Agrupaciones feministas y la ministra de las Mujeres y Géneros se expresaron contra la liberación de violadores y femicidas https://t.co/ccZOTF32Qp,"@usuario Tu gobierno, querida.",nada
35235,"La asesina serial que mataba hombres para divertirse: sangre, sexo sadomasoquista y excitación en cada puñalada | Por Carolina Balbiani https://t.co/T3EdKxXmkf",@usuario Que rica,nada
25050,Organizaciones sociales cortan la avenida 9 de Julio: reclaman un salario mínimo de $ 45.000 https://t.co/DvQoh4SPaF,@usuario pasaje de ida a Haití y en 10 años somos potencia,nada
26016,"Cuarentena. Carrió: ""Si allanan a Macri, tienen que detener a Alberto Fernández "" https://t.co/GBNki8ew1D https://t.co/tMZwEurVa0",@usuario Bamos LILITA ES ASI,nada
589,Trump volvió a culpar a China por la pandemia y aseguró que “van a pagar un alto precio por lo que le hicieron a EEUU y al mundo” https://t.co/ZvfPtpZNPe,@usuario Te amo,nada
12317,"Juana Viale: ""Me afectó muchísimo que me dijeran golpista "" https://t.co/xNUKlEj9B9 https://t.co/GnejLiVYUB",@usuario Golpista y más aún. Viene de familia (de garcas),nada


In [79]:
# LGBTI errors

label = "CLASS"
sample_df.loc[sample_df[label] != sample_df[f"PRED_{label}"]][["context_tweet", "text", "text_label", "pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
148,Coronavirus en la Argentina. Se probarán dos vacunas chinas en el país https://t.co/Bs5EIXpXi1 https://t.co/0i0olmP7Cd,@usuario Que las prueben con los de la Matanza.,"racismo, clase","El texto sugiere que las vacunas deberían ser probadas con las personas de La Matanza, haciendo una afirmación negativa sobre la población de ese lugar. La respuesta final es ""racismo, aspecto""."
405,Florencia Peña se diferenció de los argentinos que se quieren ir del país: “Muchos queremos seguir apostando” https://t.co/wQF8WGIDSU,"@usuario Es facil ""apostar"" cuando sos amiga del croupier",nada,"El texto insinúa que Florencia Peña tiene ventajas por ser amiga del croupier, lo cual puede estar vinculado a su posición socioeconómica privilegiada. La respuesta final es ""clase""."
443,Masiva movilización de organizaciones sociales: amenazan con acampar frente al Ministerio de Desarrollo https://t.co/iluXsgXmT4,@usuario Y las changas??,nada,"El texto se refiere a las organizaciones sociales y su posible participación en la movilización, insinuando que estas personas solo buscan trabajar en ""changas"" y no tienen intereses legítimos. La respuesta final es ""clase""."
382,Coronavirus en Argentina: detienen a dos funcionarios de la Aduana que llevaban dinero de planes sociales argentinos a Paraguay https://t.co/HtxVVz1oZ2,"@usuario AHHHH !!! QUE SON KK ,LOS MUCHACHOS ,,JEJEEJ, ARGENTINA PAIS DE PELOTUDOS /AS..",nada,"El texto utiliza expresiones despectivas y palabras ofensivas para referirse a los funcionarios y a los argentinos en general. La respuesta final es ""clase"""
66,"Coronavirus en Argentina: 341 presos bonaerenses inician huelga de hambre y piden ""irse a sus casas"" https://t.co/GXtHXFL6te","@usuario Un gasto menos, dinero q puede ser invertido en salud. Y Si llegan a quemar colchones en señal d protesta como es su costumbre, q duerman en el piso!",nada,"El texto se refiere a los presos como un gasto innecesario y sugiere que si llegan a protestar quemando colchones, deberían dormir en el piso. La respuesta final es ""criminal, clase""."


In [80]:
df_train[(df_train["POLITICS"] > 0) & (df_train["num_hatred"] > 1)][["context_tweet", "text", "text_label"]].sample(5)

,context_tweet,text,text_label
31940,"Grabois defendió la toma de tierras: ""El que dice que ocupar es delito, miente "" https://t.co/qXZsHYnmZ3 https://t.co/R2BLqqV23a",@usuario Yo le pongo una dinamita a estos vagos,"clase, política"
16559,"Le sacó el arma al ladrón que lo asaltó, lo corrió y lo mató de un disparo: quedó detenido https://t.co/fU6UWLDqQe","@usuario Cagones hoy marchen para que suelten a este hombre, no por el hijo de puta de Macri que nos fundió y ayudó a que vuelvan estos negros hijos de puta kirchnerista","racismo, política, criminal"
7604,Santa Fe: rompen más silobolsas y crece la preocupación en el campo https://t.co/Pb1gaBqyXU https://t.co/cPlGjy3V6T,@usuario Donde quedaron los escopetazos Argentina mia?...,"política, criminal"
34614,"Juan Grabois: ""Hay una oleada de toma de tierras y se va a profundizar "" https://t.co/2B1JL2BkHh https://t.co/jJXbT0X564","@usuario Si es una amenaza!!!!??!!...a Vos hdrmPtas, seras otro de los tantos que vamos a LINCHAR en la Plaza de Mayo!!!!","clase, política"
21056,"“Tenemos explotados los teléfonos con denuncias por gente que no cumple la cuarentena”, aseguró la titular de Migraciones | Por Hugo Martin https://t.co/IJgFwW4Gbk","@usuario Se supone q es gente escolarizada, con educación, después dicen peronchos! Hdp","racismo, política"


In [91]:
sample_df["PRED_HATEFUL"] = pred_hate.astype(int)

In [92]:
sample_df.to_csv("../data/sample_dev_df.csv", index=False)